# Project enda : Example A

In this example notebook, we will show how to read and manipulate contracts data on a small sample.
Then we will show how to align it with consumption, weather and TSO forecast data in order to train it and make a load forecast.

To start, you will need a python 3 installation (use a virtual environment), and to install some packages. You can do it using poetry, or pip.

```bash
# create virtualenv, can use for instance {path_to_python3.9} instead of just "python3"
python3 -m venv {path-to-venv}    
source {path-to-venv}/bin/activate
which python  # check python path

# install enda with poetry 
# next line install required packages to run the examples
poetry install enda

# install enda with pip
pip install enda 
# in that case, you'll also need some extra packages, 
# such as jupyter to run that notebook, or matplotlib 
pip install jupyter numexpr joblib matplotlib
```

Then you can download `example_a.zip` to your local machine. It contains  this notebook (`ExampleA.ipynb`) and the dataset (`contracts.csv`, `historic_load_measured.csv`, `weather_and_tso_forecasts.csv`). Open `ExampleA.ipynb` with jupyter and follow the tutorial there instead of the pdf/html. The dataset is a micro-example of the data we typically deal with. 

We here pretend **we are exactly on '2020-09-20'** and want to predict our SLP (synthetic load profiles) customers load for the next 3 days, from '2020-09-21' to '2020-09-23' at a 15 min time-step. The desired time-zone is 'Europe/Berlin'. This load may depend on several factors such as the number of customer or the weather. In this example, we have only 3 days of training data, from '2020-09-16' to '2020-09-19'. 

Data from '2020-09-20' is not available because we do not have the most recent measured consumption data : there is a time-gap between the latest time for which we have an actual measure and the next time we want to predict. In a more realistic example, this gap may be a few days or weeks.   


The files are :
- `contracts.csv` : contains a list of 7 electricity customer contracts with different characteristics. 
- `historic_load_measured.csv` : the past load for 2 groups of customers : `smart_metered` and `slp`, from '2020-09-16' to '2020-09-19'.
- `weather_and_tso_forecasts.csv` : 2 external forecasts, the temperature and the total load on our TSO's grid, available in the past and in the future : from '2020-09-16' to '2020-09-23'. 

You can now follow this tutorial step by step. It is divided in 3 parts:
1. Deal with contracts data
2. Make a really basic prediction
3. Try it yourself


In [ ]:
import os
import pandas as pd
import enda

In [ ]:
enda.__file__

In [ ]:
# replace with the folder path where you put example_a
DIR = '.'

## 1. Deal with contracts data

In [ ]:
contracts = enda.Contracts.read_contracts_from_file(os.path.join(DIR, "contracts.csv"))

In [ ]:
contracts
# When date_end_exclusive = NaT, this means the contract is still active today and has no planned end date.
# Note that lines 1 and 2 are about the same customer with customer_id=1. They changed their subscribed power, 
#   so we counted it as a new contract (contract_id=1-a then 1-b).
# Note that some have a start date or an end date in the 'future' (after '2020-09-20').

In [ ]:
# we are only interested in SLP customers here
contracts_slp = contracts[~contracts["smart_metered"]].copy() # drop smart-metered contracts
# add a variable to count the number of active contracts
contracts_slp["contracts_count"] = 1

In [ ]:
contracts_slp

In [ ]:
# count the running total of ["contracts_count", "subscribed_power_kva", "estimated_annual_consumption_kwh"]  each day
portfolio_slp_by_day = enda.Contracts.compute_portfolio_by_day(
    contracts_slp, 
    columns_to_sum = ["contracts_count", "subscribed_power_kva", "estimated_annual_consumption_kwh"],
    date_start_col="date_start",
    date_end_exclusive_col="date_end_exclusive"
)

In [ ]:
# note that portfolio_by_day can have dates in the future (after 2020-09-20) if some contracts have a future date_end
portfolio_slp_by_day

In [ ]:
# restrict/extend the portfolio_by_day to desired dates 
portfolio_slp_by_day = enda.PortfolioTools.get_portfolio_between_dates(
    portfolio_slp_by_day, 
    start_datetime = pd.to_datetime('2020-09-16'),
    end_datetime_exclusive = pd.to_datetime('2020-09-24')
)

In [ ]:
portfolio_slp_by_day

In [ ]:
# turn the portfolio_by_day into a portfolio timeseries with our desired freq and timezone
portfolio_slp = enda.Resample.upsample_and_interpolate(
    portfolio_slp_by_day,
    freq='15min', 
    tz_info='Europe/Berlin', 
    forward_fill=True
)

In [ ]:
portfolio_slp

## 2. Make a really basic prediction

In [ ]:
# read historical load, weather and TSO forecast data
historic_load_measured = pd.read_csv(os.path.join(DIR, "historic_load_measured.csv"))
weather_and_tso_forecasts = pd.read_csv(os.path.join(DIR, "weather_and_tso_forecasts.csv"))

In [ ]:
# correctly format 'time' as a pandas.DatetimeIndex of dtype: datetime[ns, tzinfo]
for df in [historic_load_measured, weather_and_tso_forecasts]:
    df['time'] = pd.to_datetime(df['time']) 
    # for now df['time'] can be of dtype "object" because there are 2 french timezones: +60min and +120min. 
    # it is important to align time-zone to 'Europe/Berlin' to make sure the df has a pandas.DatetimeIndex
    df['time'] = enda.TimezoneUtils.convert_dtype_from_object_to_tz_aware(df['time'], tz_info = 'Europe/Berlin')
    df.set_index('time', inplace=True)

In [ ]:
historic_load_measured

In [ ]:
weather_and_tso_forecasts

In [ ]:
# lets create the train set with historical data
portfolio_slp_historic = portfolio_slp[portfolio_slp.index <= historic_load_measured.index.max()]

slp_historic = pd.merge(
    portfolio_slp_historic, 
    historic_load_measured[['slp_kw']], 
    how='inner', left_index=True, right_index=True
)

slp_historic = pd.merge(
    slp_historic, 
    weather_and_tso_forecasts, 
    how='inner', left_index=True, right_index=True
)

slp_historic

In [ ]:
# lets create the input data for our forecast
portfolio_slp_forecast = portfolio_slp[portfolio_slp.index >= pd.to_datetime('2020-09-21 00:00:00+02:00')]

slp_forecast_input = pd.merge(
    portfolio_slp_forecast, 
    weather_and_tso_forecasts, 
    how='inner', left_index=True, right_index=True
)

slp_forecast_input

In [ ]:
# create minimalistic features, for the example, just the hour 
def featurize(df):
    df = df.copy(deep=True)
    df["hour"] = df.index.hour
    return df

In [ ]:
slp_historic = featurize(slp_historic)
slp_forecast_input = featurize(slp_forecast_input)

In this example we will use a simple linear regression using the implementation in `sklearn`.
Enda has a wrapper that works with any scikit-learn estimator : `enda.ml_backends.sklearn_estimator.EndaSklearnEstimator`. It makes it easier to deal with timeseries and pandas dataframes. It also allows to use estimators in more advanced models defined in enda. 

To save a trained model we will use `joblib`.

Install the requirements if you haven't already :
```bash
pip install scikit-learn joblib
```

In [ ]:
from enda.ml_backends.sklearn_estimator import EndaSklearnEstimator
from sklearn.linear_model import LinearRegression
import joblib

In [ ]:
lin_reg = EndaSklearnEstimator(LinearRegression())
lin_reg.train(slp_historic, target_col='slp_kw')

In [ ]:
# save model to a file 
model_path = os.path.join(DIR, "lin_reg.joblib")
joblib.dump(lin_reg, model_path)  
del lin_reg

In [ ]:
# load model from the file 
lin_reg = joblib.load(model_path)
prediction = lin_reg.predict(slp_forecast_input, target_col='slp_kw')
assert (prediction.index == slp_forecast_input.index).all()  # verify that the pandas.DatetimeIndex is conserved

In [ ]:
prediction

To visualize pandas dataframes, we use `matplotlib` as backend. Install it if you haven't already :
```bash
pip install matplotlib
```

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot consumption : historic and forecast
to_plot = pd.merge(
    slp_historic["slp_kw"].to_frame("slp_kw_historic"), 
    prediction.rename(columns={"slp_kw": "slp_kw_forecast"}), 
    how='outer', left_index=True, right_index=True
)
to_plot.plot()                             

In [ ]:
# plot the size of the portfolio of SLP customers over time
portfolio_slp[["contracts_count", "subscribed_power_kva"]].plot()

In [ ]:
# plot consumption per kva: historic and forecast
to_plot = pd.merge(
    (slp_historic["slp_kw"]/slp_historic["subscribed_power_kva"]).to_frame("slp_kw_per_kva_historic"), 
    (prediction["slp_kw"]/portfolio_slp["subscribed_power_kva"]).to_frame("slp_kw_per_kva_forecast"), 
    how='outer', left_index=True, right_index=True
)
to_plot.plot()       

## 3. Try it yourself

As an exercise, you should repeat the previous analysis/prediction but this time on `smart-metered` customers.

## Conclusion

Thats all for this introduction. Go to Example B for a more complete and in-depth example.
Thanks for reading and don't hesitate to send feeback at: team-data@enercoop.org !